# Nobelplot

## Preliminaries

In [1]:
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.glyphs import Text
from collections import OrderedDict
import csv
import math

In [10]:
plotwidth = 700
boundary = 50

def triple2ternary(triple, pw=800):
    # normalize 
    triple = [float(i)/sum(triple) for i in triple]
    y = triple[2] * math.sin(math.radians(60));
    x = 1 - triple[0] - y * 1/math.tan(math.radians(60));
    y = 1-y
    return pw*x,pw*y

def citation2size(cit):
    return int(plotwidth/440 * (round(math.sqrt(400.*float(cit)))+4))
    #return int(float(abs(cit))*100 + 10)

In [11]:
# https://stackoverflow.com/questions/19486369/extract-csv-file-specific-columns-to-list-in-python/19487003
with open('../data/transdiagnosis_sig.csv', 'r', encoding='utf-8-sig') as infile:
  # read the file as a dictionary for each row ({header : value})
  reader = csv.DictReader(infile)
  datain = {}
  for row in reader:
    for header, value in row.items():
      try:
        datain[header].append(value.replace(u'\xa0', u' '))
      except KeyError:
        datain[header] = [value.replace(u'\xa0', u' ')]

datain['x'] = [triple2ternary([float(i),float(j),float(k)], plotwidth)[0] for i,j,k in zip(datain['MDD-HC'], datain['BIP-HC'], datain['SCZ-HC'])]
datain['y'] = [triple2ternary([float(i),float(j),float(k)], plotwidth)[1] for i,j,k in zip(datain['MDD-HC'], datain['BIP-HC'], datain['SCZ-HC'])]
datain['color'] = ['#F4BB41' if i == "Transdiagnosis" else '#081F5C' if i == "MDD-Specific" else '#4EAD5B' if i == "BIP-Specific" else '#B02418' for i in datain['Discipline']]
#datain['color'] = ['#D4433D' if i == "CONa" else '#D4433D' if i == "CONb" else '#D4433D' if i == "CONc" else '#5B89BB' if i == "DMNa" else '#5B89BB' if i == "DMNb" else '#5B89BB' if i == "DMNc" else '#75B567' if i == "DANa" else '#75B567' if i == "DANb" else '#F1923C' if i == "LMN" else '#DAD07B' if i == "SVNa" else '#DAD07B' if i == "SVNb" else '#A56A45' if i == "SMNa" else '#A56A45' if i == "SMNb" else '#A56A45' if i == "TempPar" else '#E993C3' if i == "VSNc" else '#E993C3' if i == "VSNp" else '#A3A3A3' for i in datain['Discipline']]
#datain['colordark'] = ['#205483' if i == "within-CON" else '#96932c' if i == "between-CON" else '#b26d24' for i in datain['Discipline']]
datain['size'] = [citation2size(i) for i in datain['Combined ES']]
datain['titletruncated'] = [(i[:65] + '..') if len(i) > 65 else i for i in datain['Title']]

source = ColumnDataSource(data=dict(
    x = datain['x'],
    y = datain['y'],
    color = datain['color'],
    #colordark = datain['colordark'],
    size = datain['size'],
    discipline = datain['Discipline'],
    publicationyear = datain['FC Number'],
    c_MDD = datain['MDD-HC'],
    c_BIP = datain['BIP-HC'],
    c_SCZ = datain['SCZ-HC'],
    c_total = datain['Combined ES'],
    title = datain['titletruncated'],
))

In [12]:
# https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html
hover = HoverTool(names=["papers"], tooltips="""
    <div style="margin-bottom: 10px;font-size: 12px">
        <span style="">@title</br></span>
    </div>
    <div style="font-size: 12px;font-weight: bold">
        Effect Size: @c_total = <span style="color: #205483">@c_MDD </span> + <span style="color: #b26d24">@c_BIP </span> + <span style="color: #96932c">@c_SCZ </span>
    </div>
    """
)

In [13]:
nobelplot = figure(title="The effect size of dorsal attention network", tools=[hover,"wheel_zoom","pan","box_zoom","reset"], toolbar_location="right",
            x_range=[0-boundary, plotwidth+boundary], y_range=[0-0.2*boundary, plotwidth+1.8*boundary], active_drag="pan", active_scroll="wheel_zoom",plot_width=plotwidth, plot_height=plotwidth)

nobelplot.patch([plotwidth*0.1,plotwidth*0.9,plotwidth*0.5,plotwidth*0.1], [plotwidth,plotwidth,plotwidth-0.8*plotwidth*math.sqrt(3)/2,plotwidth], alpha=1, line_width=0, color="#ede4ea")
nobelplot.line(x=[0,plotwidth,0.5*plotwidth,0], y=[plotwidth,plotwidth,plotwidth-plotwidth*math.sqrt(3)/2,plotwidth], line_color="#1a1919", line_width=2, line_alpha=1)
nobelplot.circle('x', 'y', size="size", fill_color="color", name="papers", fill_alpha= 0.92, line_color="#1a1919", line_width=1.5, source=source)

source2 = ColumnDataSource(dict(x=[boundary/2+70,plotwidth-boundary/2-45,plotwidth/2], y=[plotwidth+boundary/3,plotwidth+boundary/3,plotwidth-1.09*plotwidth*math.sqrt(3)/2], text=["MDD - HC","BIP - HC","SCZ - HC"], cols=["#205483","#b26d24","#96932c"], text_align=["left","right","center"]))
glyph = Text(x="x", y="y", text="text", text_color="cols", text_align="center", text_font_size="18pt" )
nobelplot.add_glyph(source2, glyph)

nobelplot.axis.visible = False
nobelplot.xgrid.visible = False
nobelplot.ygrid.visible = False
#nobelplot.outline_line_color = None

output_file("../nobelplot.html", title="Nobel plot")
show(nobelplot)  # open a browser